<a href="https://colab.research.google.com/github/AlexCHEN-Engineer/CEE314HW/blob/main/CEE314Problemsets4Q4Teaching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%time
# Do not revise here. 
try:
    import google.colab
except ImportError:
    import ufl
    import dolfin
    import mshr
else:
    try:
        import ufl
        import dolfin
        import mshr
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
        import ufl
        import dolfin
        import mshr

# Variational formulation

In this part, we will solve a coupled poromechanics problems.

For a domain denoted as $\Omega$, the boundary is $\Gamma = \Gamma_u\cup \Gamma_t = \Gamma_p\cup\Gamma_q$, $\Gamma_u\cap \Gamma_t=\phi$ and $\Gamma_p\cup\Gamma_q=\phi$. 

The strong form can be expressed as:
\begin{equation}
\begin{aligned}
&\nabla(\boldsymbol{\sigma}^\prime-p\boldsymbol{1})+\rho\boldsymbol{g}=\boldsymbol{0},   \\
&\nabla⋅\dot{\boldsymbol{u}}+\nabla⋅{\boldsymbol{q}}=0, \\
& \boldsymbol{u} = \overline{\boldsymbol{u}}\quad on \quad \Gamma_u, \\
& \boldsymbol{\sigma}^\prime\cdot\boldsymbol{n} = \overline{\boldsymbol{t}}\quad on\quad \Gamma_t, \\
& p = \overline{p}\quad on \quad \Gamma_p, \\
& \boldsymbol{q}\cdot \boldsymbol{n} = \overline{q}\quad on \quad \Gamma_q.
\end{aligned}
\end{equation}

As for the constitutive equations, we need to refer to
\begin{equation}
\begin{aligned}
& \boldsymbol{\sigma}^\prime = \mathbb{C}^e:\boldsymbol{\epsilon} ,\\
& \boldsymbol{q} = -\frac{\boldsymbol{\kappa}}{\mu_f}\cdot \nabla p.
\end{aligned}
\end{equation}


## Derive the weak form

The trial space is defined as 
\begin{equation}
\begin{aligned}
& S_u = \{\boldsymbol{u}:\Omega→\mathbb{R}^{ndim}|\boldsymbol{u}→\boldsymbol{H}^1,\ \boldsymbol{u}=\overline{\boldsymbol{u}}\quad on \quad \Gamma_u \} ,\\
&S_p = \{p:\Omega→\mathbb{R}|p→H^1,\ p=\overline{p}\quad on \quad \Gamma_p \}
\end{aligned}
\end{equation}

For the weighting space,
\begin{equation}
\begin{aligned}
& V_u = \{\boldsymbol{\eta}:\Omega→\mathbb{R}^{ndim}|\boldsymbol{\eta}→\boldsymbol{H}^1,\ \boldsymbol{\eta}={\boldsymbol{0}}\quad on \quad \Gamma_u \} ,\\
&V_\psi = \{\psi:\Omega→\mathbb{R}|\psi→H^1,\ \psi=0\quad on \quad \Gamma_\psi \}
\end{aligned}
\end{equation}


The weak form becomes:
\begin{equation}
\begin{aligned}
& \mathcal{G}_1 = \int_{\Omega}(\nabla^s\boldsymbol{\eta}:\boldsymbol{\sigma}^\prime-p\nabla\cdot\boldsymbol{\eta}-\boldsymbol{\eta}\cdot{\rho}\boldsymbol{g})dV - \int_{\Gamma_t}\boldsymbol{\eta}\cdot\overline{\boldsymbol{t}}dA = \boldsymbol{0} \\
& \mathcal{G}_2 = \int_{\Omega}(\psi\nabla\cdot\dot{\boldsymbol{u}}-\nabla \psi\cdot \boldsymbol{q})dV + \int_{\Gamma_q} \psi\bar{q}dA\\
\end{aligned}
\end{equation}

# Import necessary library

In [ ]:
from fenics import *
from mshr import *
from ufl import nabla_div

In [ ]:
# Create mesh and define expression
# mesh = RectangleMesh.create([Point(0, 0), Point(1,1)],[2,2],CellType.Type.quadrilateral)
# We can still create the mesh here.
# But, we can't use plot to see the results.

# Parameters

In this part, please use the following mechanical and hydraulic parameters. 
* $\lambda = 0,\quad \mu = 500$
* $k = 1.0\times 10^{-11},\quad \mu_l = 1.0\times 10^{-2}$
All the parameters adopt the [international units](https://en.wikipedia.org/wiki/International_unit).

In [ ]:
#####################
#### TODO ###########
#####################
########## Start your code here ###########
T = None          # final time
num_steps = None  # number of time steps
dt = None         # time step size
rho_l = None      # fluid density
E = None          # Young's modulus
nu = None         # poisson's ratio
k = None   	      # intrinsic permeability
mu_l = None 	  # dynamic viscosity of IF
lambda_= None     # Lame coefficient
mu = None         # Lame coefficient
########### End your code here #############

In [ ]:
# Create mesh and define expression
mesh=RectangleMesh(Point(0.0,0.0), Point(10., 5.), 10, 5,'crossed')
# mesh = RectangleMesh.create([Point(0, 0), Point(10, 5)], [20, 10], CellType.Type.quadrilateral)
top_u =  CompiledSubDomain("near(x[1], side) && x[0] > -DOLFIN_EPS && x[0] < 2.+DOLFIN_EPS", side = 5.)
top_p =  CompiledSubDomain("near(x[1], side)", side = 5.)
bottom =  CompiledSubDomain("near(x[1], side)", side = 0.0)
left_right = CompiledSubDomain("(near(x[0], 0.0) ) || (near(x[0], 10.) )")

boundary = MeshFunction('size_t', mesh, mesh.topology().dim()-1)
boundary.set_all(1)
top_u.mark(boundary, 2)
ds = Measure('ds', domain = mesh, subdomain_data = boundary)

T  = Expression(('0.0', '-10'),degree=1)  # Load on the boundary

#Define Mixed Space (R2,R) -> (u,p)
V = VectorElement("CG", mesh.ufl_cell(), 2)
W = FiniteElement("CG", mesh.ufl_cell(), 1)
L = FunctionSpace(mesh,W)
MS = dolfin.FunctionSpace(mesh, MixedElement([V,W]))

# Define boundary condition
bcu1 = DirichletBC(MS.sub(0).sub(0), 0.0, left_right)  # slip condition
bcu2 = DirichletBC(MS.sub(0).sub(1), 0.0, bottom)      # slip condition
bcp = DirichletBC(MS.sub(1), 0.0, top_p)               # drained condition
bc=[bcu1,bcu2,bcp]

# Define strain.
def epsilon(u):
    return 0.5*(nabla_grad(u) + nabla_grad(u).T)
# define stress.
def sigma(u):
   return lambda_*tr(epsilon(u))*Identity(2) + 2.0*mu*epsilon(u)

# Define variational problem and initial condition
X0 = Function(MS)
B = TestFunction(MS)

e_u0 = Expression(('0.0', '0.0'), degree=1)
e_p0 = Expression('0.0', degree=1)

u0 = interpolate(e_u0, MS.sub(0).collapse())
p0 = interpolate(e_p0, MS.sub(1).collapse())

Xn = Function(MS)
assign(Xn, [u0, p0])

(u,p)=split(X0)
(u_n,p_n)=split(Xn)
(v,q)=split(B)


#####################
#### TODO ###########
#####################
############### Start your code here ###########
F = None 
############### End your code here #############

#solver tuning
dX0 = TrialFunction(MS)
J = derivative(F, X0, dX0)
Problem = NonlinearVariationalProblem(F, X0, J = J, bcs = bc)
Solver  = NonlinearVariationalSolver(Problem)
Solver.parameters['newton_solver']['convergence_criterion'] = 'incremental'
Solver.parameters['newton_solver']['relative_tolerance'] = 1.e-11
Solver.parameters['newton_solver']['absolute_tolerance'] = 5.e-10

#if you want to store the results
vtkfile_u = File('MC/u.pvd')
vtkfile_p = File('MC/p.pvd')

t = 0

p_out = []

from tqdm import tqdm
for n in tqdm(range(1)):
	t += dt
	Solver.solve()    
	assign(Xn,X0)

	(_u,_p)=X0.split()
	
	vtkfile_u << (_u,t)
	vtkfile_p << (_p,t)
	p_out.append(_p(0., 3.))

<class 'ufl.indexed.Indexed'>


100%|██████████| 1/1 [00:00<00:00, 29.11it/s]


In [ ]:
u_result, p_result = X0.split()
import matplotlib.pyplot as plt
c=plot(p_result)
plt.colorbar(c)
plt.show()

# Postprocess

In [ ]:
# Use the following method to zip the file.
!zip -r 'MC.zip' '/content/MC' 

## 4(b)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
time = np.linspace(0,1000,1000)
pressure = np.array(p_out)
plt.xlabel(r"$t$")
plt.ylabel(r"$p/w$")
plt.semilogx(100/9.8*1e-3*time/4, pressure/10, 'k')